In [22]:
import websocket
import json
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import sympy as sp
import scipy as sc
from time import sleep
from binance import Client
client = Client(api_key="jM9zEeTTzT7R7QGlNrh5tUpOc8l71hOxu7CewCL4M0NvbBp3ilgmEwMOOjJfCRVB",api_secret="iZxFmlz5bvROsHQPZTP5hlJtsxYjOu8wocOlLWzIDdhMzEQu7KyyHB2DseOYQ66s",testnet=True)

# from google.cloud.sql.connector import Connector
# from sqlalchemy import create_engine, text
# import pg8000.native

# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

# connector = Connector()

# def getconn() -> pg8000.native.Connection:
#     conn: pg8000.native.Connection = connector.connect(
#         "semiotic-pact-388619:us-central1:tradingbot",
#         "pg8000",
#         user="postgres",
#         password="GuruSat.3",
#         db="criptos"
#     )
#     return conn

# connection = create_engine('postgresql+pg8000://',creator=getconn)

In [23]:
client.get_account(recvWindow=10000)

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712666256119,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '0.99700000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9993.85563610', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '737.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.00000000

In [4]:
datetime.utcnow()

datetime.datetime(2024, 4, 9, 10, 34, 10, 247067)

In [27]:
############################################################################################################################################################
# INICIO DEL BOT DE TRADING.
############################################################################################################################################################



##################################################
# PARÁMETROS INICIALES.
##################################################
###################################
# Parámetros de interpolación.
###################################
smooth_interval = 60
smooth_exp = 3

###################################
# Parámetros de estrategia.
###################################
rango = 20
std_mult = 150
pdi_mult = 25
sdi_mult = 250
z_aprox = 0.01

###################################
# Metadatos de la estrategia.
###################################
estrategia = 'DERIVATIVES_TESTNET_V1'
trade_qty = 0.001
k = 0
bullish = False

###################################
# Data Frames y Listas de Trabajo.
###################################
recent_data = pd.DataFrame(columns=['Tiempo','Precio'])
buy_orders = []
sell_orders = []
max_points_idx = []
min_points_idx = []

############################################################################################################################################################
# Ciclo anterior al streaming para obtener mejores LOGS y menos código innecesario.
#############################################################################################################################################################
while True:
    if k == 0:
        print(f'Inicia el bot de trading con inicio en: {datetime.utcnow()}')
    else:
        if bullish == True:
            print(f'Se realizó una compra cerca del: {datetime.utcnow()}')
        else:
            print(f'Se realizó una venta cerca del: {datetime.utcnow()}')
            
    k += 1

    ##################################################################################################################################
    # INICIA BOT DE TRADING: ACTUALIZACIÓN DE DATOS, CÁLCULO DE DERIVADAS, ESTRATEGIA Y TRADES AUTOMÁTICOS.
    ##################################################################################################################################
    #############################################
    # Inicio del streaming
    #############################################

    
    #############################################
    # Limpieza y almacenamiento de los datos de streaming en el df correspondiente.
    #############################################
    recent_data = pd.read_csv('../../../csvs/streaming/BTCUSDT.csv',skiprows=120 + k)
    recent_data.columns = ['Tiempo','Simbolo','Precio']
    recent_data['Tiempo'] = recent_data.Tiempo.astype('datetime64')
    recent_data = recent_data.set_index('Tiempo')
    recent_data['Precio'] = recent_data.Precio.astype(float)
    recent_data = recent_data.reset_index()
    print(f'Punto 1: {datetime.utcnow()}')

    pmax_idx = 0
    pmin_idx = 0
    last_time_buy = recent_data.Tiempo[0]
    last_time_sell = recent_data.Tiempo[0]
    
    ##################################################################################################################################
    # CÁLCULO DE DERIVADAS, ESTRATEGIA Y TRADES AUTOMÁTICOS.
    ##################################################################################################################################
    if len(recent_data) > 60:
        print(f'Punto 2: {datetime.utcnow()}')
        if len(recent_data) > 120:
            recent_data = recent_data[-120:]
        
        #############################################
        # CÁLCULO DE DERIVADAS
        #############################################
        recent_data['Interpolacion'] = sc.signal.savgol_filter(recent_data.Precio,smooth_interval,smooth_exp)
        recent_data['Primer_Derivada'] = recent_data.Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
        recent_data['PD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Primer_Derivada,smooth_interval,smooth_exp)
        recent_data['Segunda_Derivada'] = recent_data.PD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
        recent_data['SD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Segunda_Derivada,smooth_interval,smooth_exp)
        recent_data['Tercer_Derivada'] = recent_data.SD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
        recent_data['TD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Tercer_Derivada,smooth_interval,3)
        recent_data = recent_data.dropna()
        lista_min_max = recent_data[(recent_data['TD_Interpolacion'] >= -z_aprox) & (recent_data['TD_Interpolacion'] <= z_aprox)]
        indices_min_max = lista_min_max.index.tolist()
        mp = recent_data.Precio.mean()
        
        print(f'Punto 3: {datetime.utcnow()}')

        #############################################
        # ESTRATEGIA
        #############################################
        pmin = 0
        pmax = 0

        for PMSD in indices_min_max:
            if PMSD > len(recent_data) - rango:
                break
            if (recent_data.SD_Interpolacion[PMSD:PMSD+1].item() > 0) & (recent_data.PD_Interpolacion[PMSD:PMSD+1].item() > 0):
                pmax_idx = PMSD
                if ((mp + recent_data.SD_Interpolacion[PMSD]*sdi_mult) > (mp + np.std(recent_data.dropna.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD]*pdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)):
                    pmax = recent_data.SD_Interpolacion[PMSD]
                for i in range(rango+1):
                    try:
                        if ((recent_data.SD_Interpolacion[PMSD-i] > pmax)) & ((mp + recent_data.SD_Interpolacion[PMSD-i]*sdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)):
                            pmax_idx = PMSD-i
                        elif ((recent_data.SD_Interpolacion[PMSD+i] > pmax)) & ((mp + recent_data.SD_Interpolacion[PMSD+i]*sdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD+i]*pdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)):
                            pmax_idx = PMSD+i
                    except:
                        pass
                    try:
                        max_points_idx.append(pmax_idx)
                    except:
                        pass
            elif (recent_data.SD_Interpolacion[PMSD:PMSD+1].item() < 0) & (recent_data.PD_Interpolacion[PMSD:PMSD+1].item() < 0):
                pmin_idx = PMSD
                if ((mp + recent_data.SD_Interpolacion[PMSD]*sdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD]*pdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)):
                    pmin = recent_data.SD_Interpolacion[PMSD]
                for i in range(rango+1):
                    try:
                        if ((recent_data.SD_Interpolacion[PMSD-i] < pmin)) & ((mp + recent_data.SD_Interpolacion[PMSD-i]*sdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)):
                            pmin_idx = PMSD-i
                        elif ((recent_data.SD_Interpolacion[PMSD+i] < pmin)) & ((mp + recent_data.SD_Interpolacion[PMSD+i]*sdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD+i]*pdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)):
                            pmin_idx = PMSD+i
                    except:
                        pass
                    try:
                        min_points_idx.append(pmin_idx)
                    except:
                        pass
        max_points_idx = list(set(max_points_idx))
        min_points_idx = list(set(min_points_idx))
        max_points_idx.sort()
        min_points_idx.sort()
        
        print(f'Punto 4: {datetime.utcnow()}')
        
        #############################################
        # AUTOMATIC TRADING.
        #############################################
        if bullish == False:
            try:
                if (recent_data.Tiempo[max_points_idx[-1]] != last_time_buy) & (recent_data.Tiempo[max_points_idx[-1]] > last_time_buy) & (recent_data.Tiempo[max_points_idx[-1]] > last_time_sell):
                    last_time_buy = recent_data.Tiempo[max_points_idx[-1]]
                    print(f'Precio de compra en: {recent_data.Precio[max_points_idx[-1]]}. Tiempo de compra: {recent_data.Tiempo[max_points_idx[-1]]}. Tiempo real de la señal: {datetime.utcnow()}. Diferencia entre señal y compra: {(datetime.utcnow() - recent_data.Tiempo[max_points_idx[-1]]).seconds} segundos.')
                    marketorder_buy = client.create_order(
                        symbol='BTCUSDT',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=trade_qty,
                        recvWindow=15000)
                    bullish = True
                    buy_orders.append(marketorder_buy)
                    print(f'Punto 5.C: {datetime.utcnow()}')
                    # bearish = False
            except:
                pass
        else:
            try:
                if (recent_data.Tiempo[min_points_idx[-1]] != last_time_sell) & (recent_data.Tiempo[min_points_idx[-1]] > last_time_sell) & (recent_data.Tiempo[min_points_idx[-1]] > last_time_buy):
                    last_time_sell = recent_data.Tiempo[min_points_idx[-1]]
                    print(f'Precio de venta en: {recent_data.Precio[min_points_idx[-1]]}. Tiempo de venta: {recent_data.Tiempo[min_points_idx[-1]]}. Tiempo real de la señal: {datetime.utcnow()}. Diferencia entre señal y compra: {(datetime.utcnow() - recent_data.Tiempo[min_points_idx[-1]]).seconds} segundos.')
                    marketorder_sell = client.create_order(
                        symbol='BTCUSDT',
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=trade_qty,
                        recvWindow=15000)
                    sell_orders.append(marketorder_sell)
                    bullish = False
                    print(f'Punto 5.V: {datetime.utcnow()}')
                    #bearish = True
            except:
                pass

############################################################################################################################################################
# FIN DEL BOT
############################################################################################################################################################

Inicia el bot de trading con inicio en: 2024-04-09 15:23:40.281807
Punto 1: 2024-04-09 15:23:40.291808
Punto 2: 2024-04-09 15:23:40.291808
Punto 3: 2024-04-09 15:23:40.296809
Punto 4: 2024-04-09 15:23:40.296809
Se realizó una venta cerca del: 2024-04-09 15:23:40.296809
Punto 1: 2024-04-09 15:23:40.300807
Punto 2: 2024-04-09 15:23:40.300807
Punto 3: 2024-04-09 15:23:40.305807
Punto 4: 2024-04-09 15:23:40.305807
Se realizó una venta cerca del: 2024-04-09 15:23:40.305807
Punto 1: 2024-04-09 15:23:40.308813
Punto 2: 2024-04-09 15:23:40.309837
Punto 3: 2024-04-09 15:23:40.314834
Punto 4: 2024-04-09 15:23:40.314834
Se realizó una venta cerca del: 2024-04-09 15:23:40.314834
Punto 1: 2024-04-09 15:23:40.317814
Punto 2: 2024-04-09 15:23:40.317814
Punto 3: 2024-04-09 15:23:40.323348
Punto 4: 2024-04-09 15:23:40.323348
Se realizó una venta cerca del: 2024-04-09 15:23:40.323348
Punto 1: 2024-04-09 15:23:40.326347
Punto 2: 2024-04-09 15:23:40.326347
Punto 3: 2024-04-09 15:23:40.332347
Punto 4: 2024

KeyError: 0

In [14]:
recent_data

,2024-04-09 15:11:48.856,BTCUSDT,68594.08
0,2024-04-09 15:11:49.896,BTCUSDT,68559.50
1,2024-04-09 15:11:50.889,BTCUSDT,68538.86
2,2024-04-09 15:11:51.898,BTCUSDT,68556.00
3,2024-04-09 15:11:52.818,BTCUSDT,68570.28
4,2024-04-09 15:11:53.861,BTCUSDT,68560.59
...,...,...,...
364,2024-04-09 15:17:54.183,BTCUSDT,68954.58
365,2024-04-09 15:17:55.379,BTCUSDT,68954.59
366,2024-04-09 15:17:56.443,BTCUSDT,68967.51
367,2024-04-09 15:17:57.303,BTCUSDT,68967.52


In [33]:
marketorder_buy = client.create_order(
                        symbol='XRPETH',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.003,
                        recvWindow=10000)
marketorder_buy

# marketorder_buy ----> Diccionario: symbol, orderId, transactTime, orig_Qty, side, workingTime, fills ----> Lista con diccionarios. Keys: price, qty, comission, comissionaAsset, tradeId

{'symbol': 'XRPUSDT',
 'orderId': 4156737,
 'orderListId': -1,
 'clientOrderId': 'o4H8gQaUWCFIfR0cZgZlK0',
 'transactTime': 1712666256119,
 'price': '0.00000000',
 'origQty': '10.00000000',
 'executedQty': '10.00000000',
 'cummulativeQuoteQty': '6.36400000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1712666256119,
 'fills': [{'price': '0.63640000',
   'qty': '10.00000000',
   'commission': '0.00000000',
   'commissionAsset': 'XRP',
   'tradeId': 266819}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [26]:
marketorder_sell = client.create_order(
                        symbol='ETHUSDT',
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.003,
                        recvWindow=10000)
marketorder_sell

{'symbol': 'ETHUSDT',
 'orderId': 8795137,
 'orderListId': -1,
 'clientOrderId': 'mGuWjb2npgk5KI8u2UdGVQ',
 'transactTime': 1712665283539,
 'price': '0.00000000',
 'origQty': '0.00300000',
 'executedQty': '0.00300000',
 'cummulativeQuoteQty': '10.88259000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1712665283539,
 'fills': [{'price': '3627.53000000',
   'qty': '0.00300000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 1818593}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [27]:
client.get_account(recvWindow=10000)

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712665283539,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '0.99700000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.21963610', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '727.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.0000000

In [ ]:
# balances = []
# ganancia_maxima = 0
# open_order = False
# balance_trade_anterior = 0
# balance_trade_p = 0
# tiempo_inicial = datetime.utcnow()
# #### PARÁMETROS DE LAS CONDICIONES DE SALIDA Y ENTRADA.
# PAIR = 'BTCUSDT'
# pair = 'btc'
# LOCALIDAD_SEÑAL_ENTRADA = 30 # EN SEGUNDOS
# # El primero de ellos es el promedio de la pendiente (precio/tiempo) de las últimas 10 filas del dataframe. Se utiliza para 
# # saber si se debe entrar en una posición corta al alza (puesto que a la baja no estamos tradeando) junto con otra condición
# PENDIENTE_ENTRADA = 0.02
# # LOS UMBRALES SON EN DÓLARES.
# UMBRAL_PERDIDA_1P = - 5
# UMBRAL_GANANCIA_1G = 20
# UMBRAL_GANANCIA_3G = 35
# UMBRAL_GANANCIA_4G = 5
# UMBRAL_MAXIMO_2G = 20
# UMBRAL_MINIMO_2G = 5
# LOCALIDAD_SEÑAL_SALIDA_2G = 30 # EN SEGUNDOS
# LIMITE_TIEMPO_POSITIVO_4G = 10 # EN MINUTOS
# log_path = f'../LOGS/{estrategia}.csv'
# df_logs = pd.DataFrame(columns=['Tiempo','Accion'])




# ##############  VERIFICAMOS SI NO HAY OPERACIONES ABIERTAS: Como solo vamos a trabajar con señales de entrada, toda posición
# ##############  está cerrada si la siguiente condición se cumple.

#     if open_order == False:

# ##############  EN CASO DE NO HABER, SE PROCEDE A VERIFICAR LA "LOCALIDAD" DE LA SEÑAL: En caso de ser una señal local se crea 
# ##############  una orden de compra y se inicia el proceso.

#         # pendientes = [0]

#         # for i in range(len(df)-1):
#         #     x = slopee(df.index[i],df.Precio[i],df.index[i+1],df.Precio[i+1])
#         #     pendientes.append(x)

#         # df['pendiente'] = pendientes

#         DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - TIEMPO_SEÑAL_ENTRADA).total_seconds()

#         if DIFF_SEÑAL_ENTRADA < LOCALIDAD_SEÑAL_ENTRADA: # and df.pendiente[-10:-1].mean() > PENDIENTE_ENTRADA:
#             buy_order = client.create_order(
#                         symbol=PAIR,
#                         side=Client.SIDE_BUY,
#                         type=Client.ORDER_TYPE_MARKET,
#                         quantity=cantidad_trade,
#                         recvWindow=60000
#                     )
#             open_order = True
#             tiempo_entrada = datetime.utcnow()
#             precio_entrada = float(buy_order['fills'][0]['price'])
#             print(f'Se realizó una compra al precio: {precio_entrada}, en el momento: {tiempo_entrada}.')

# ##############  SI EXISTE UNA OPERACIÓN ABIERTA, LA SIGUIENTE CONDICIÓN SE CUMPLE: En ese caso, se debe estar atento a cuándo
# ##############  cerrar nuestra posición. Ya sea que estemos ganando o perdiendo; se establecen límites de pérdida y de ganancia,
# ##############  así como también casos en los que las señales de venta se toman en cuenta, tanto en pérdidas como en ganancias.

#     while open_order:

#         day = datetime.utcnow() - timedelta(seconds=1000)

#         with connection.connect() as conn:
#             df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
#         df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
#         df = df.sort_values(by='Tiempo')
#         MACD(df)
#         buy, sell = señal_MACD(df)

#         buyprices = df.Precio.iloc[buy]
#         sellprices = df.Precio.iloc[sell]
#         TIEMPO_SEÑAL_SALIDA = sellprices.index[-1]
#         precio_actual = df.Precio[-1]

#         profit = (precio_actual - precio_entrada)

#         if profit == 0:
#             continue 

# ##############  SEÑALES DE VENTA EN CASO DE TENER PÉRDIDAS.

# #############  La primera señal se enfoca en no superar el umbral de pérdidas.       

#         elif profit < 0:

#             if open_order:

#                 if profit < UMBRAL_PERDIDA_1P:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '1P'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)
                        
# ##############  SEÑALES DE VENTA EN CASO DE TENER GANANCIAS.

#         else:

# #############  La primera señal se enfoca en no perder todas las ganancias, en caso de existir ganancias. Se permite una pérdida
# #############  del 50% de la ganancia máxima.


#             if open_order:

#                 if profit > UMBRAL_GANANCIA_1G:

#                     if profit >= ganancia_maxima:
#                         ganancia_maxima = profit
#                     elif profit < ganancia_maxima:
#                         if (profit/ganancia_maxima - 1) < - 0.2: 
#                             sell_order = client.create_order(
#                                         symbol=PAIR,
#                                         side=Client.SIDE_SELL,
#                                         type=Client.ORDER_TYPE_MARKET,
#                                         quantity=cantidad_trade,
#                                         recvWindow=60000
#                                     )
#                             open_order = False
#                             Metodo = '1G'
#                             tiempo_salida = datetime.utcnow()
#                             precio_salida = float(sell_order['fills'][0]['price'])
#                             balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                             balances.append(balance_trade_p)

# #############  La segunda señal se enfoca en vender en caso de alcanzar ganancias mínimas del 0.07% y a la vez se da una señal
# #############  de venta.   

#             if open_order:     

#                 if profit <= UMBRAL_MAXIMO_2G and profit > UMBRAL_MINIMO_2G and (datetime.utcnow() - TIEMPO_SEÑAL_SALIDA).total_seconds() < LOCALIDAD_SEÑAL_SALIDA_2G:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '2G'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)
                    
# #############  La tercer señal se enfoca en vender en caso de alcanzar ganancias máximas del 1.4%

#             if open_order:
            
#                 if profit > UMBRAL_GANANCIA_3G:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '3G'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)

# ############# POR ÚLTIMO, VAMOS A GENERAR UNA VENTA SI EL TRADE ESTÁ EN GANANCIAS PERO NO SE HA VENDIDO DESPUÉS DE 10 MINUTOS.

#             if open_order:
            
#                 if ((datetime.utcnow() - tiempo_entrada).total_seconds()/60) > LIMITE_TIEMPO_POSITIVO_4G and profit > UMBRAL_GANANCIA_4G:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '4G'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)

######################## LOGS

    # if balance_trade_anterior != balance_trade_p:
    #     balance_trade_anterior = balance_trade_p
    #     if balance_trade_p>0:
    #         print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Ganancia de: {balance_trade_p} dólares. \nMétodo: {Metodo}')
    #         trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p),Metodo,LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
    #                                     columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
    #         if os.path.exists(path+f'trades_ganados_{pair}.csv'):
    #             trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
    #                             mode='a',
    #                             index=False,
    #                             header=False
    #                             )
    #         else:
    #             trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
    #                             index=False,
    #                             header=True,
    #                             )
    #     elif balance_trade_p==0:
    #         print('No hubo pérdidas ni ganancias del trade')
    #     else:
    #         print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Pérdida de: {balance_trade_p} dólares. \nMétodo: {Metodo}')
    #         trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p),Metodo,LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
    #                                     columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
    #         if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
    #             trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
    #                             mode='a',
    #                             index=False,
    #                             header=False
    #                             )
    #         else:
    #             trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
    #                             index=False,
    #                             header=True,
    #                             )

In [3]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/logs/Sesiones/MACD_MACD+logicv1/'

# sleep(5)
    
if open_order:
    sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=cantidad_trade
                        )
    open_order = False
    precio_salida = float(sell_order['fills'][0]['price'])
    balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
    balances.append(balance_trade_p)
    
if tiempo_entrada > tiempo_salida:
    tiempo_final = tiempo_entrada
elif tiempo_entrada < tiempo_salida:
    tiempo_final = tiempo_salida 

if sum(balances) > 0:
    session_log = pd.DataFrame([[len(balances), sum(balances), sum(balances)/len(balances), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
             columns=['Cantidad_Trades','Ganancia','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
    if os.path.exists(path+'Sesiones_Ganadas.csv'):
        session_log.to_csv(path+'Sesiones_Ganadas.csv',
                        mode='a',
                        index=False,
                        header=False
                        )
    else:
        session_log.to_csv(path+'Sesiones_Ganadas.csv',
                        index=False,
                        header=True
                        )
elif sum(balances) < 0:
    session_log = pd.DataFrame([[len(balances), sum(balances), sum(balances)/len(balances), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
             columns=['Cantidad_Trades','Perdida','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
    if os.path.exists(path+'Sesiones_Perdidas.csv'):
        session_log.to_csv(path+'Sesiones_Perdidas.csv',
                        mode='a',
                        index=False,
                        header=False
                        )
    else:
        session_log.to_csv(path+'Sesiones_Perdidas.csv',
                        index=False,
                        header=True
                        )

In [4]:
session_log

,Cantidad_Trades,Ganancia,Ganancia_Promedio_xTrade,PAR,Estrategia,Tiempo_inicial,Tiempo_final,Cantidad_trade
0,44,0.14105,0.003206,BTCUSDT,MACD_logicv1,2023-07-05 13:55:44.163789,2023-07-05 14:33:14.443281,0.005


In [12]:
# balances[-1] = balances[-1]*0.1
balances

[0.08279999999998836,
 0.07739999999997962,
 0.05650000000001455,
 -0.05930000000000291,
 0.057000000000007274,
 0.20440000000002329]

In [28]:
sum(balances) + 1.5547999999999957

-0.09275000000008782

In [4]:
client.get_account()

BinanceAPIException: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.

In [38]:
sum(balances)

-55.60899999998987

In [117]:
pd.to_datetime('2023-06-29 20:03:37.435685') - pd.to_datetime('2023-06-29 19:05:24.924680')

Timedelta('0 days 00:58:12.511005')

In [121]:
balances[-1]

-0.7869999999998982

In [39]:

buy_order = client.create_order(
                        symbol='BTCUSDT',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )

In [32]:
sell_order = client.create_order(
                            symbol='BTCUSDT',
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=0.1
                        )

In [16]:
sell_order

{'symbol': 'BTCUSDT',
 'orderId': 10464206,
 'orderListId': -1,
 'clientOrderId': 'zmNXnF77mLQvDiTWZ94hZS',
 'transactTime': 1688165636752,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '3046.84891893',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1688165636752,
 'fills': [{'price': '30468.63000000',
   'qty': '0.00859600',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2990498},
  {'price': '30468.55000000',
   'qty': '0.06432900',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2990499},
  {'price': '30468.30000000',
   'qty': '0.02707500',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2990500}],
 'selfTradePreventionMode': 'NONE'}

In [5]:
marketorder_buy = client.create_order(
    symbol='BTCUSDT',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=0.1,
    recvWindow=10000)

In [16]:
marketorder_sell = client.create_order(
    symbol='BTCUSDT',
    side=Client.SIDE_SELL,
    type=Client.ORDER_TYPE_MARKET,
    quantity=0.1,
    recvWindow=10000)

In [17]:
limitorder_sell

{'symbol': 'BTCUSDT',
 'orderId': 15582800,
 'orderListId': -1,
 'clientOrderId': 'UbJC3pf8YldNgFrnFPYmys',
 'transactTime': 1712660072002,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '7057.64658920',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1712660072002,
 'fills': [{'price': '70620.43000000',
   'qty': '0.00574000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611441},
  {'price': '70620.30000000',
   'qty': '0.00390000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611442},
  {'price': '70618.14000000',
   'qty': '0.00100000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611443},
  {'price': '70614.61000000',
   'qty': '0.00100000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611444},
  {'price': '70613.49000000',
   'qty': '0.00393000',
   'commi

In [18]:
client.get_all_orders(symbol='BTCUSDT',recvWindow=10000) 15609234

order = client.get_order(
    symbol='BTCUSDT',
    orderId='15582800',
    recvWindow=10000)

In [19]:
order

{'symbol': 'BTCUSDT',
 'orderId': 15582800,
 'orderListId': -1,
 'clientOrderId': 'UbJC3pf8YldNgFrnFPYmys',
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '7057.64658920',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1712660072002,
 'updateTime': 1712660072002,
 'isWorking': True,
 'workingTime': 1712660072002,
 'origQuoteOrderQty': '0.00000000',
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [10]:
cancel_order = client.cancel_order(
    symbol='BTCUSDT',
    orderId='10459808')

BinanceAPIException: APIError(code=-2011): Unknown order sent.

In [15]:
limitorder

{'symbol': 'BTCUSDT',
 'orderId': 10459808,
 'orderListId': -1,
 'clientOrderId': 'Vgl1y7Y4unFF40r881ULls',
 'transactTime': 1688164704216,
 'price': '30800.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '3051.23838466',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'workingTime': 1688164704216,
 'fills': [{'price': '30512.38000000',
   'qty': '0.08717800',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 2989981},
  {'price': '30512.41000000',
   'qty': '0.01282200',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 2989982}],
 'selfTradePreventionMode': 'NONE'}

In [92]:
sell_order

{'symbol': 'BTCUSDT',
 'orderId': 9914874,
 'orderListId': -1,
 'clientOrderId': 'n3IbYFnRv5COlykPamCSvW',
 'transactTime': 1688061602684,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '3054.32647239',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1688061602684,
 'fills': [{'price': '30543.33000000',
   'qty': '0.02747100',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2793732},
  {'price': '30543.24000000',
   'qty': '0.05140300',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2793733},
  {'price': '30543.24000000',
   'qty': '0.02112600',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2793734}],
 'selfTradePreventionMode': 'NONE'}

In [20]:
client.get_account(recvWindow=10000)

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712660072002,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.90000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '17057.64658920', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '727.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.0000000

In [34]:
client.get_all_orders(symbol='BTCUSDT',recvWindow=10000) 15609234

[{'symbol': 'BTCUSDT',
  'orderId': 15582800,
  'orderListId': -1,
  'clientOrderId': 'UbJC3pf8YldNgFrnFPYmys',
  'price': '0.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.10000000',
  'cummulativeQuoteQty': '7057.64658920',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1712660072002,
  'updateTime': 1712660072002,
  'isWorking': True,
  'workingTime': 1712660072002,
  'origQuoteOrderQty': '0.00000000',
  'selfTradePreventionMode': 'EXPIRE_MAKER'},
 {'symbol': 'BTCUSDT',
  'orderId': 15609234,
  'orderListId': -1,
  'clientOrderId': 'E9ilsoPQPBOt3ypxACHUE7',
  'price': '0.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.10000000',
  'cummulativeQuoteQty': '7068.30954310',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1712663642250,
  'updateTime': 17126636